In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
import orbitfit.utils as utils
from orbitfit.orbitfit import run_fit
import dateutil.parser
import copy
import json
import orbdetpy
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import numpy as np

%matplotlib notebook

In [ ]:
start = dateutil.parser.parse("2018-11-12 00:00")
end = dateutil.parser.parse("2018-11-12 08:00")
sate = "newsat5"
df_gps = utils.download_gps_data(sate, start, end)
df_gps_eci = utils.rotate_gps_to_eci(df_gps)

In [ ]:
config_dict = {'Gravity': {'Degree': 70, 'Order': 70},
                   'OceanTides': {'Degree': 20, 'Order': 20},
                   'Drag': {'Model': 'MSISE', 'Coefficient': {'Value': 2.0}},
                   'SolidTides': {'Sun': True, 'Moon': True},
                   'ThirdBodies': {'Sun': True, 'Moon': True},
                   'RadiationPressure': {'Sun': True, 'Creflection': {'Value': 1.5}},
                   'SpaceObject': {'Mass': 40.0, 'Area': 0.8 * 0.4}}
df_out, df_gps_eci_interpolated, config_dict_out = run_fit(df_gps_eci, config_dict, max_loops=2)

In [ ]:
((df_gps_eci_interpolated - df_out)[["randv_mks_{}".format(i) for i in range(3)]]).plot()
plt.title("Pos error [m]")

In [ ]:
config_dict_out

In [ ]:
dates = []
pos_vel = []
with open("stk_result_newsat5_20181112_2.txt", 'r') as fid:
    for line in fid.readlines()[7:]:
        try:
            data = line.split("  ")
            dates.append(dateutil.parser.parse(data[0]).replace(tzinfo=datetime.timezone.utc))
            pos_vel.append([float(value)*1000 for value in data[1:] if value is not ""])
        except Exception:
            print(data)
            break
df_stk = pd.DataFrame(data=pos_vel, index=dates, columns=[f"randv_mks_{i}" for i in range(6)])

In [ ]:
((df_stk - df_out)[["randv_mks_{}".format(i) for i in range(3)]]).plot()
plt.title("STK - orbdetpy [m]")

In [ ]:
pos_diff_ecef_m = (df_stk - df_out)[["randv_mks_{}".format(i) for i in range(3)]]
pos_stk_ecef_m = df_stk[["randv_mks_{}".format(i) for i in range(3)]].values
teme2arr = utils.find_inertial2aar_dcm(pos_stk_ecef_m)
pos_diff_aar_m = np.einsum("ijk,ik->ij", teme2arr, pos_diff_ecef_m)
plt.figure()
plt.plot(df_stk.index, pos_diff_aar_m, '-')
plt.grid()
plt.title("STK - orekit  diff")
plt.ylabel("diff [km]")
plt.legend(["along-track", "cross-track", "radial"])
plt.tight_layout()
plt.show()

La propagación de STK se adelanta respecto a la de orekit. Sospecho que el problema esta en el drag. STK estima un drag mayor, por lo que cae y se adelanta. Voy a intentar variar el valor de drag hasta que coincidan los propagadores

In [ ]:
config_dict_out

Variando el area

In [ ]:
config_dict_copy = copy.deepcopy(config_dict_out)
config_dict_copy["SpaceObject"]["Area"] = 0.320*8
output = orbdetpy.simulateMeasurements(json.dumps(config_dict_copy))
output_dict = json.loads(output)
index = pd.DatetimeIndex([o["Time"] for o in output_dict])
data = [[float(x) for x in o["TrueState"]["Cartesian"][:6]] for o in output_dict]
df_out = pd.DataFrame(data=data, index=index, columns=["randv_mks_{}".format(i) for i in range(6)])

In [ ]:
pos_diff_ecef_m = (df_stk - df_out)[["randv_mks_{}".format(i) for i in range(3)]]
pos_stk_ecef_m = df_stk[["randv_mks_{}".format(i) for i in range(3)]].values
teme2arr = utils.find_inertial2aar_dcm(pos_stk_ecef_m)
pos_diff_aar_m = np.einsum("ijk,ik->ij", teme2arr, pos_diff_ecef_m)
plt.figure()
plt.plot(df_stk.index, pos_diff_aar_m, '-')
plt.grid()
plt.title("STK - orekit  diff")
plt.ylabel("diff [m]")
plt.legend(["along-track", "cross-track", "radial"])
plt.tight_layout()
plt.show()

Variando el CD

In [ ]:
config_dict_copy = copy.deepcopy(config_dict_out)
config_dict_copy["Drag"]["Coefficient"]["Value"] = 15.5
output = orbdetpy.simulateMeasurements(json.dumps(config_dict_copy))
output_dict = json.loads(output)
index = pd.DatetimeIndex([o["Time"] for o in output_dict])
data = [[float(x) for x in o["TrueState"]["Cartesian"][:6]] for o in output_dict]
df_out = pd.DataFrame(data=data, index=index, columns=["randv_mks_{}".format(i) for i in range(6)])

In [ ]:
pos_diff_ecef_m = (df_stk - df_out)[["randv_mks_{}".format(i) for i in range(3)]]
pos_stk_ecef_m = df_stk[["randv_mks_{}".format(i) for i in range(3)]].values
teme2arr = utils.find_inertial2aar_dcm(pos_stk_ecef_m)
pos_diff_aar_m = np.einsum("ijk,ik->ij", teme2arr, pos_diff_ecef_m)
plt.figure()
plt.plot(df_stk.index, pos_diff_aar_m, '-')
plt.grid()
plt.title("STK - orekit  diff")
plt.ylabel("diff [m]")
plt.legend(["along-track", "cross-track", "radial"])
plt.tight_layout()
plt.show()

Why STK drag is much bigger than orbdetpy? No idea!